# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn_pandas
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')

## Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.



F1 es el que mejor se adapta? ver bien jeje

En nuestro caso utilizaremos Acuracy porque funciona muy bien para los problemas de clasificación, y ademas nos permite observar aquellos datos con mayor precisión
Esta tecnica nos va a poder mostrar el porcentaje de aciertos que tuvimos.

In [2]:
# Importamos el dataset.
dataset = pd.read_csv('Video_Games_Sales.csv',encoding='UTF-8')
dataset.sample(25)

,Unnamed: 0,Game,Platform,Gener,Publisher,North_America,Europe,Japan,Rest_of _World,Global,Good_Sale,Count_Games
8288,8288,Samurai Deeper Kyo,PS,Fighting,Namco Bandai Games,0.00,0.00,0.05,0.00,0.05,0,765
131,131,ATV Offroad Fury 3,PS2,Racing,SouthPeak Games,1.00,0.04,0.00,0.16,1.20,2,30
9751,9751,Tetris Party Deluxe,DS,Puzzle,Nintendo,0.61,0.46,0.14,0.12,1.32,0,665
5983,5983,Mister Mosquito,PS2,Shooter,Eidos Interactive,0.05,0.04,0.17,0.01,0.28,0,121
6483,6483,NFL Fever 2002,XB,Sports,Microsoft Game Studios,0.74,0.21,0.00,0.04,0.99,0,178
11144,11144,World Class Track Meet,NES,Sports,Namco Bandai Games,1.92,0.45,0.64,0.07,3.08,2,765
7834,7834,RealSports Volleyball,2600,Sports,Atari,0.12,0.01,0.00,0.00,0.13,0,277
4925,4925,Kimi no Yuusha,DS,Role-Playing,SNK Playmore,0.00,0.00,0.01,0.00,0.01,0,16
3394,3394,Fuse (Insomniac),X360,Shooter,Electronic Arts,0.15,0.07,0.00,0.03,0.24,0,598
11363,11363,Yu-Gi-Oh! 5D's World Championship 2011: Over t...,DS,Strategy,Konami Digital Entertainment,0.10,0.03,0.07,0.01,0.21,0,637


In [3]:
dataset.dtypes

Unnamed: 0          int64
Game               object
Platform           object
Gener              object
Publisher          object
North_America     float64
Europe            float64
Japan             float64
Rest_of _World    float64
Global            float64
Good_Sale           int64
Count_Games         int64
dtype: object

In [4]:
# Limpiamos un poco el dataset
dataset.drop(columns=['Unnamed: 0', 'Platform','North_America', 'Europe', 'Japan', 'Rest_of _World'],axis=1,inplace=True)
dataset.dtypes

Game            object
Gener           object
Publisher       object
Global         float64
Good_Sale        int64
Count_Games      int64
dtype: object

In [5]:
# Transformamos la variable Gener a One-Hot Encoding
dataset.Gener.value_counts()

Gener
Action          1949
Sports          1377
Misc            1304
Role-Playing    1227
Adventure       1060
Shooter          825
Racing           771
Simulation       726
Fighting         611
Platform         588
Strategy         580
Puzzle           498
Name: count, dtype: int64

In [6]:
dummies = pd.get_dummies(dataset['Gener'], drop_first = True)
dummies.head()

,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False


In [7]:
## Como quedo en booleano (true o false), se tranforma a binario
dummies = dummies.astype({'Adventure': 'int32', 'Fighting': 'int32'
                         ,'Misc': 'int32', 'Platform': 'int32'
                         ,'Puzzle': 'int32', 'Racing': 'int32'
                         ,'Role-Playing': 'int32', 'Shooter': 'int32'
                         ,'Simulation': 'int32', 'Sports': 'int32'
                         ,'Strategy': 'int32'})
dummies.dtypes

Adventure       int32
Fighting        int32
Misc            int32
Platform        int32
Puzzle          int32
Racing          int32
Role-Playing    int32
Shooter         int32
Simulation      int32
Sports          int32
Strategy        int32
dtype: object

In [8]:
dummies.head()

,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0


In [9]:
# Añadimos las variables al DataSet
dataset = pd.concat([dataset, dummies], axis = 1)

# Eliminamos la vairable original Gener
dataset = dataset.drop(columns=['Gener'])

In [10]:
dataset.sample(25)

,Game,Publisher,Global,Good_Sale,Count_Games,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
5690,McGrath Vs. Pastrana Freestyle Motocross,Acclaim Entertainment,0.26,0,114,0,0,0,0,0,1,0,0,0,0,0
5997,Mobile Suit Gundam Seed Destiny: Rengou vs. Z....,Namco Bandai Games,0.49,0,765,0,0,0,0,0,0,0,1,0,0,0
11481,ZooCube,Acclaim Entertainment,0.07,0,114,0,0,0,0,1,0,0,0,0,0,0
5545,Malice,Evolved Games,0.03,0,4,0,0,0,1,0,0,0,0,0,0,0
7319,Playmobil Circus,Mindscape,0.09,0,27,0,0,0,0,0,0,0,0,0,0,0
4149,Hitman 2: Silent Assassin,Eidos Interactive,4.28,2,121,0,0,0,0,0,0,0,0,0,0,0
5226,Line Rider 2: Unbound,Deep Silver,0.10,0,69,0,0,0,0,1,0,0,0,0,0,0
10546,True Crime: New York City,Activision,1.21,0,401,0,0,0,0,0,0,0,0,0,0,0
3313,Fox Sports Golf '99,Gremlin Interactive Ltd,0.14,0,9,0,0,0,0,0,0,0,0,0,1,0
9329,Super Dodge Ball Advance,Ubisoft,0.20,0,564,0,0,0,0,0,0,0,0,0,1,0


In [11]:
dataset = dataset.rename(columns={'Role-Playing':'RolePlaying'})

In [12]:
dataset.sample(5)

,Game,Publisher,Global,Good_Sale,Count_Games,Adventure,Fighting,Misc,Platform,Puzzle,Racing,RolePlaying,Shooter,Simulation,Sports,Strategy
10519,Trigger Man,Play It,0.23,0,13,0,0,0,0,0,0,0,1,0,0,0
6325,NASCAR 06: Total Team Control,Electronic Arts,0.90,0,598,0,0,0,0,0,1,0,0,0,0,0
7464,Power Rangers: Wild Force,THQ,0.21,0,394,0,0,0,0,0,0,0,0,0,0,0
110,A.C.E.: Another Century's Episode,Banpresto,0.26,0,71,0,0,0,0,0,0,0,0,1,0,0
8453,Senran Kagura: Bon Appétit!,Marvelous Entertainment,0.03,0,11,0,0,1,0,0,0,0,0,0,0,0


## Se debe aplicar alguna técnica de feature engineering para mejorar los datos de entrada a los modelos, y mostrar la comparativa de los resultados obtenidos en cada caso. Si no es posible o útil, fundamentar el motivo por el cual no se realizará.



In [13]:
mapper = DataFrameMapper([
    (['Count_Games'], [StandardScaler()]),
    (['Adventure'], [OneHotEncoder()]),
    (['Fighting'], [OneHotEncoder()]),
    (['Misc'], [OneHotEncoder()]),
    (['Platform'], [OneHotEncoder()]),
    (['Puzzle'], [OneHotEncoder()]),
    (['Racing'], [OneHotEncoder()]),
    (['RolePlaying'], [OneHotEncoder()]),
    (['Shooter'], [OneHotEncoder()]),
    (['Simulation'], [OneHotEncoder()]),
    (['Sports'], [OneHotEncoder()]),
    (['Strategy'], [OneHotEncoder()])
])

In [14]:
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(dataset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

((6909, 16), (2303, 16), (2304, 16))

In [15]:
# Función para evaluar modelos con la tecnica de future engineering
def evaluar_Modelo(model, set_names=('train', 'validation'), Titulo=''):
    display(Titulo)
        
    final_metrics = {
        'Precision': [],
        'Recall': [],
        'F1': [], 
    }
    ##final_metrics = {'Accuracy': []}
   
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name] 
        y = set_data.Good_Sale
        y_pred = model.predict(set_data)
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred, average='micro'))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred, average='micro'))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred, average='micro'))
        ##final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

In [16]:
#Con feature engeeniring, sin hiperparámetros.

# kNN
Modelo_KNN = Pipeline([('mapper', mapper), ('classifier', KNeighborsClassifier(n_neighbors=10))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN')

'kNN'

,Precision,Recall,F1
train,0.842958,0.842958,0.842958
validation,0.827182,0.827182,0.827182


In [17]:
# Decision Tree
Modelo_Decision_Tree = Pipeline([('mapper', mapper), ('classifier', DecisionTreeClassifier(random_state=42))])
Modelo_Decision_Tree.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree, Titulo='Decision Tree')

'Decision Tree'

,Precision,Recall,F1
train,0.853524,0.853524,0.853524
validation,0.823708,0.823708,0.823708


In [18]:
# Gradient Boosting
Modelo_Gradient_Boosting = Pipeline([('mapper', mapper), ('classifier', GradientBoostingClassifier(random_state=42))])
Modelo_Gradient_Boosting.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting, Titulo='Gradient Boosting')

'Gradient Boosting'

,Precision,Recall,F1
train,0.844695,0.844695,0.844695
validation,0.832393,0.832393,0.832393


Segun los datos obtenidos, podemos observar que el modelo Gradient Boosting posee mejor resultado en validations en comparacion con los otros 2 modelos.

En el caso de train, el de mejor resultado es Decision Tree

## Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.

Evaluaremos los modelos con los Hiper-parámetros: 
* Para KNN:
    * n_neighbors: 10, 20 y 30
    * weights: distance - uniform
    * p:1 (distancia de Manhattan) - 2 (distancia Euclidiana)
* Para Decision Tree
    * max_depth: 10, 20 y 30
    * min_samples_split: 10
    * min_samples_leaf: 10
* Gradient Bosting
    * max_depth de 10, 20 y 30
    * n_estimators: 10
    * learning_rate: 0.5
    * max_features: sqrt   

In [19]:
# kNN 10/uniform/1
Modelo_KNN = Pipeline([('mapper', mapper), 
                       ('classifier', KNeighborsClassifier(n_neighbors=10, weights='uniform', p=1))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 10/uniform/1')

# kNN 20/uniform/1
Modelo_KNN = Pipeline([('mapper', mapper), 
                       ('classifier', KNeighborsClassifier(n_neighbors=20, weights='uniform', p=1))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 20/uniform/1')

# kNN 30/uniform/1
Modelo_KNN = Pipeline([('mapper', mapper), 
                       ('classifier', KNeighborsClassifier(n_neighbors=30, weights='uniform', p=1))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 30/uniform/1')

# kNN 10/distance/2
Modelo_KNN = Pipeline([('mapper', mapper), 
                       ('classifier', KNeighborsClassifier(n_neighbors=10, weights='distance', p=2))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 10/distance/2')

# kNN 20/distance/2
Modelo_KNN = Pipeline([('mapper', mapper),
                       ('classifier', KNeighborsClassifier(n_neighbors=20, weights='distance', p=2))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 20/distance/2')

# kNN 30/distance/2
Modelo_KNN = Pipeline([('mapper', mapper), 
                       ('classifier', KNeighborsClassifier(n_neighbors=30, weights='distance', p=2))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN 30/distance/2')

'kNN 10/uniform/1'

,Precision,Recall,F1
train,0.843103,0.843103,0.843103
validation,0.825879,0.825879,0.825879


'kNN 20/uniform/1'

,Precision,Recall,F1
train,0.843248,0.843248,0.843248
validation,0.829353,0.829353,0.829353


'kNN 30/uniform/1'

...

,Precision,Recall,F1
train,0.850195,0.850195,0.850195
validation,0.822406,0.822406,0.822406


'kNN 20/distance/2'

,Precision,Recall,F1
train,0.851064,0.851064,0.851064
validation,0.823274,0.823274,0.823274


'kNN 30/distance/2'

,Precision,Recall,F1
train,0.853524,0.853524,0.853524
validation,0.825011,0.825011,0.825011


En el modelo KNN podemos notar que, con weights = uniform y p = 1, a medida que aumenta el valor de n_neighbors, aumenta el resultado de validations.

En el caso de weights = distance y p = 2, se observa lo mismo, solo que el aumento se produce con valores menos distantes entre si.

In [20]:
# Decision Tree 10
Modelo_Decision_Tree_10 = Pipeline([('mapper', mapper), 
                                   ('classifier', DecisionTreeClassifier(max_depth = 10, min_samples_split=10, min_samples_leaf=10))])
Modelo_Decision_Tree_10.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_10, Titulo='Decision Tree 10')

# Decision Tree 20
Modelo_Decision_Tree_20 = Pipeline([('mapper', mapper), 
                                    ('classifier', DecisionTreeClassifier(max_depth = 20, min_samples_split=10, min_samples_leaf=10))])
Modelo_Decision_Tree_20.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_20, Titulo='Decision Tree 20')

# Decision Tree 30
Modelo_Decision_Tree_30 = Pipeline([('mapper', mapper), 
                                    ('classifier', DecisionTreeClassifier(max_depth = 30, min_samples_split=10, min_samples_leaf=10))])
Modelo_Decision_Tree_30.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_30, Titulo='Decision Tree 30')

'Decision Tree 10'

,Precision,Recall,F1
train,0.844985,0.844985,0.844985
validation,0.833695,0.833695,0.833695


'Decision Tree 20'

,Precision,Recall,F1
train,0.845419,0.845419,0.845419
validation,0.831524,0.831524,0.831524


'Decision Tree 30'

,Precision,Recall,F1
train,0.845419,0.845419,0.845419
validation,0.831524,0.831524,0.831524


En el caso de Decision Tree, podemos notar que a medida que incrementamos el parametro de max_depth, los resultados no cambian, sa diferencia solo se observa con valores pequeños de max_depth.

In [21]:
# Gradient Boosting 10
Modelo_Gradient_Boosting_10 = Pipeline([('mapper', mapper), 
                                        ('classifier', GradientBoostingClassifier
                                         (max_depth = 10, random_state=42, n_estimators=10, 
                                          learning_rate=0.5, max_features='sqrt'))])
Modelo_Gradient_Boosting_10.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_10, Titulo='Gradient Boosting 10')

# Gradient Boosting 20
Modelo_Gradient_Boosting_20 = Pipeline([('mapper', mapper), 
                                        ('classifier', GradientBoostingClassifier
                                         (max_depth = 20, random_state=42, n_estimators=10, 
                                          learning_rate=0.5, max_features='sqrt'))])
Modelo_Gradient_Boosting_20.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_20, Titulo='Gradient Boosting 20')

# Gradient Boosting 30
Modelo_Gradient_Boosting_30 = Pipeline([('mapper', mapper), 
                                        ('classifier', GradientBoostingClassifier
                                         (max_depth = 30, random_state=42, n_estimators=10, 
                                          learning_rate=0.5, max_features='sqrt'))])
Modelo_Gradient_Boosting_30.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_30, Titulo='Gradient Boosting 30')

'Gradient Boosting 10'

,Precision,Recall,F1
train,0.852945,0.852945,0.852945
validation,0.825011,0.825011,0.825011


'Gradient Boosting 20'

,Precision,Recall,F1
train,0.853524,0.853524,0.853524
validation,0.818063,0.818063,0.818063


'Gradient Boosting 30'

,Precision,Recall,F1
train,0.853524,0.853524,0.853524
validation,0.813287,0.813287,0.813287


En Gradient Boosting vemos que a medida que aumenta el valor de max_depth, el valor obtenido disminuye.

## Realizar experimentos que utilicen como datos de entrada representaciones intermedias de los datos (generadas por técnicas de reducción de dimensiones como PCA). Compare los resultados obtenidos contra los casos previos, interprete y proponga conclusiones.

Aplicaremos el PCA con uno de cada modelo anterior, eligiendo el que mejor resultado nos dio.

In [22]:
# KNN-PCA-10/distance/2
Modelo_KNN = Pipeline([('mapper', mapper), ('pca', PCA()),
                       ('classifier', KNeighborsClassifier(n_neighbors=10, weights='distance', p=2))])
Modelo_KNN.fit(train, train.Good_Sale)
predictions = Modelo_KNN.predict(validation)
evaluar_Modelo(Modelo_KNN, Titulo='kNN PCA 10/distance/2')

'kNN PCA 10/distance/2'

,Precision,Recall,F1
train,0.850195,0.850195,0.850195
validation,0.823274,0.823274,0.823274


In [23]:
# Decision Tree PCA 20
Modelo_Decision_Tree_20 = Pipeline([('mapper', mapper), ('pca', PCA()),
                                    ('classifier', DecisionTreeClassifier(max_depth = 20, min_samples_split=10, min_samples_leaf=10))])
Modelo_Decision_Tree_20.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Decision_Tree_20, Titulo='Decision Tree 20')

'Decision Tree 20'

,Precision,Recall,F1
train,0.847156,0.847156,0.847156
validation,0.825879,0.825879,0.825879


In [24]:
# Gradient Boosting PCA 30
Modelo_Gradient_Boosting_30 = Pipeline([('mapper', mapper), ('pca', PCA()),
                                        ('classifier', GradientBoostingClassifier
                                         (max_depth = 30, random_state=42, n_estimators=10, 
                                          learning_rate=0.5, max_features='sqrt'))])
Modelo_Gradient_Boosting_30.fit(train, train.Good_Sale)
evaluar_Modelo(Modelo_Gradient_Boosting_30, Titulo='Gradient Boosting 30')

'Gradient Boosting 30'

,Precision,Recall,F1
train,0.853524,0.853524,0.853524
validation,0.814590,0.814590,0.814590


Despues de agregar el PCA, podemos observar que en Decision Tree, el valor obtenido baja un poco, mientras que en el caso de KNN y Gradient Boosting, el valor aumenta un poco.

## Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar y considerar no solo el rendimiento del modelo en su elección, sino también cuestiones como interpretabilidad, tiempos de entrenamiento, etc.



El modelo que utilizaremos para informarle al cliente va a ser Decision Tree con una profundidad maxima de 5 y con PCA.

Este modelo fue elegido porque fue el que menos diferencia entre los valores optenidos en train y validation, como tambien por ser el que mejor resultado nos mostro con el valor de validation.

El valor final que informaremos sera de 83,23%